In [1]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

np.random.seed(0)

Kuzushiji-MNIST is a drop-in replacement for the MNIST dataset (28x28 grayscale, 70,000 images), provided in the original MNIST format as well as a NumPy format. Since MNIST restricts us to 10 classes, we chose one character to represent each of the 10 rows of Hiragana when creating Kuzushiji-MNIST.

In [2]:
ds, ds_info = tfds.load('kmnist', as_supervised=True, with_info=True)
ds_info

Shuffling and writing examples to /root/tensorflow_datasets/kmnist/3.0.1.incompleteQPZD6T/kmnist-test.tfrecord


  0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset kmnist downloaded and prepared to /root/tensorflow_datasets/kmnist/3.0.1. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='kmnist',
    version=3.0.1,
    description='Kuzushiji-MNIST is a drop-in replacement for the MNIST dataset (28x28 grayscale, 70,000 images), provided in the original MNIST format as well as a NumPy format. Since MNIST restricts us to 10 classes, we chose one character to represent each of the 10 rows of Hiragana when creating Kuzushiji-MNIST.',
    homepage='http://codh.rois.ac.jp/kmnist/index.html.en',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@online{clanuwat2018deep,
      author       = {Tarin Clanuwat and Mikel Bober-Irizar and Asanobu Kitamoto and Alex Lamb and Kazuaki Yamamoto and David Ha},
      title        = {Deep Learning for Classical Japanese Literature},
      date 

In [4]:
tfds.as_dataframe(ds['train'].take(10), ds_info)

,image,label
0,,9 (wo)
1,,1 (ki)
2,,7 (ya)
3,,2 (su)
4,,1 (ki)
5,,2 (su)
6,,1 (ki)
7,,8 (re)
8,,8 (re)
9,,6 (ma)


In [5]:
def normalize(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

In [6]:
ds_train = ds['train'].map(normalize).batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds['test'].map(normalize).batch(128).prefetch(tf.data.AUTOTUNE)
ds_train.element_spec

(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [20]:
def define_model(filters, kernel_size, input_shape, pool_size, nodes):
  """Based on a model by Francois Chollet https://github.com/fchollet/deep-learning-with-python-notebooks
  """
  model = Sequential()  # model is a linear stack of layers
  #convolutional layers and dense layers require an activation function
  model.add(Conv2D(filters,
                   kernel_size=kernel_size,
                   padding='valid',
                   input_shape=input_shape,
                   activation='relu'))  #first conv. layer  KEEP
  model.add(Conv2D(filters,
                   kernel_size=kernel_size,
                   padding='valid',
                   activation='relu'))  #2nd conv. layer  KEEP

  model.add(MaxPooling2D(pool_size=pool_size)) #decreases size, prevent overfitting
  model.add(Dropout(0.5))  #zeros out some fraction of inputs, prevent overfitting
  model.add(Flatten())  #must flatten before going into conventional dense layer  KEEP
  print('Model flattened out to ', model.output_shape)

  #now start a typical neural network
  model.add(Dense(nodes, activation='relu'))  #change neurons in this layer
  model.add(Dropout(0.5))
  model.add(Dense(n_classes, activation='softmax'))  #10 final nodes (one for each class)  KEEP
  #softmax at end to pick between classes 0-9 KEEP

  # many optimizers available, see https://keras.io/optimizers/#usage-of-optimizers
  # suggest you KEEP loss at 'categorical_crossentropy' for multi-class problem
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer = Adam(learning_rate=0.001),
                metrics=['sparse_categorical_accuracy']) #multi-label integers (not OHE)
  return model

In [21]:
#a small batch size is key
batch_size = 32  #number of training samples used at a time to update the weights
n_classes = 10    #number of output possibilities: [0 - 9] KEEP
epochs = 6   #number of passes through the entire train dataset before weights "final"
input_shape = (28, 28, 1)   # px x px x 1 channel image input (grayscale) KEEP
filters = 24    #number of convolutional filters to use
pool_size = (2, 2)  #pooling decreases image size, reduces computation, adds translational invariance
kernel_size = (4, 4)  #convolutional kernel size, slides over image to learn features
nodes = 512  #neurons in dense layer

model = define_model(filters, kernel_size, input_shape, pool_size, nodes)
# during fit process watch train and test error simultaneously
model.fit(ds_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(ds_test))

score = model.evaluate(ds_test, verbose=0)
print('Test cross-entropy score:', score[0])
print('Test cross-entropy accuracy:', score[1])  # this is the one we care about

Model flattened out to  (None, 2904)
Epoch 1/6
469/469 [==============================] - 6s 13ms/step - loss: 0.3786 - sparse_categorical_accuracy: 0.8812 - val_loss: 0.3162 - val_sparse_categorical_accuracy: 0.9053
Epoch 2/6
469/469 [==============================] - 6s 13ms/step - loss: 0.1414 - sparse_categorical_accuracy: 0.9571 - val_loss: 0.2451 - val_sparse_categorical_accuracy: 0.9278
Epoch 3/6
469/469 [==============================] - 6s 13ms/step - loss: 0.1010 - sparse_categorical_accuracy: 0.9688 - val_loss: 0.1984 - val_sparse_categorical_accuracy: 0.9432
Epoch 4/6
469/469 [==============================] - 6s 12ms/step - loss: 0.0827 - sparse_categorical_accuracy: 0.9741 - val_loss: 0.1825 - val_sparse_categorical_accuracy: 0.9481
Epoch 5/6
469/469 [==============================] - 5s 11ms/step - loss: 0.0704 - sparse_categorical_accuracy: 0.9773 - val_loss: 0.1699 - val_sparse_categorical_accuracy: 0.9529
Epoch 6/6
469/469 [==============================] - 6s 12ms/st